## Transformers architecture를 받아와 구축해보자!

Transformers 강의 + 실습까지 수강하시느라 너무 수고 많으셨습니다! 저도 저번 기수 당시 transformers를 처음 접했는데, 모델의 구조가 난해하고 쓰인 개념들도 어려워서 이해하는데 많은 시간이 걸렸었네요.. 그래도 transformers 모델이 현재 contemporary AI technology에 쓰이지 않는 곳이 없다보니, 어려운 내용들이 다수 있지만 힘주어서 중요한 내용들로 채워넣으려고 노력했습니다 수고하셨습니다 XD

Transformers은 개념도 개념이지만, 매번 새롭게 attention 코드를 짜고, encoder와 decoder 구조를 구축하는 것도 막막하실 겁니다! 다행히도 transformers architecture는 워낙 유명해서 이제 코드 몇줄만 `딸깍`해도 최신 논문기반 transformer 구조를 `huggingface` 혹은 `github`에서 받아서 사용할 수 있습니다 :D 이번 실습에는 초심자가 사용하기는 어렵지만 `x-transformers`에서 저희가 구축한 transformers 구조를 받아와, 예시 문장을 출력하는 것까지 마무리할 예정입니다!

transformers architecture가 2017년 나온 이후로, 이 architecture를 기반으로 한 여러 모델들이 나왔고, 또한 base model에 대해서도 여러 개선점들이 추가되었습니다. `x-transformers`은 이 개선된 model들을 코드 몇줄을 추가해서 적용 가능하게 하는 라이브러리로, 최신 논문 동향을 파악하고 있어야 한다는 점에서 어렵지만 그만큼 성능이 뒷받침해주는 코드들을 모아둔 라이브러리입니다.

TMI가 생각보다 길어졌는데, 아무튼 이번 과제는 따로 작성해 넣어야할 부분은 없고, 제가 드리는 코드를 그대로 실행하기만 하면 되는 과정으로 추가했습니다. 이제 개강이 얼마 남지 않았는데, 다들 화이팅입니다 XD

의현 그는 신인가

# Colab GPU 환경에서 구동하세요!

In [1]:
!pip install x-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.5 MB/s eta 0:00:00


In [2]:
import torch
from x_transformers import XTransformer

## Transformers Architectures 받아오
model = XTransformer(
    ## 모델의 차원 (논문 = 512)
    dim = 16,
    ## encoder token의 개수 (논문 = 256)
    enc_num_tokens = 16,
    ## encoder 반복 횟수 (논문 = 6)
    enc_depth = 6,
    ## multihead attention n_heads 개수 (논문 = 8)
    enc_heads = 8,
    ## encoder token의 max sequence length (논문 = 1024)
    enc_max_seq_len = 32,
    ## (논문 = 256)
    dec_num_tokens = 16,
    dec_depth = 6,
    dec_heads = 8,
    ## (논문 = 1024)
    dec_max_seq_len = 32,
    tie_token_emb = True
).cuda()

In [3]:
## NUM_BATCHES = Epoches의 개수
## BATCH_SIZE = 하나의 batch에 들어갈 sample의 개수
## LEARNING_RATE = learning rate
## GENERATE_EVERY = 100번마다 한번씩 generate해서 accuracy확인
## NUM_TOKENS = 데이터 내 유니크한 토큰의 수
## ENC_SEQ_LEN = encoder sequence length

NUM_BATCHES = int(1e3)
BATCH_SIZE = 32
LEARNING_RATE = 3e-4
GENERATE_EVERY  = 100
NUM_TOKENS = 4 + 2
ENC_SEQ_LEN = 8
DEC_SEQ_LEN = 16 + 1

In [4]:
## Transformer 모델에 넣을 임의의 src, tgt, src_mask 생성

def cycle():
    while True:
        prefix = torch.ones((BATCH_SIZE, 1)).long().cuda()
        src = torch.randint(2, NUM_TOKENS, (BATCH_SIZE, ENC_SEQ_LEN)).long().cuda()
        tgt = torch.cat((prefix, src, src), 1)
        src_mask = torch.ones(BATCH_SIZE, src.shape[1]).bool().cuda()
        yield (src, tgt, src_mask)

In [5]:
## Train Model

import tqdm
import torch.optim as optim

optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

## loss update 해가면서 학습
for i in tqdm.tqdm(range(NUM_BATCHES), mininterval=10., desc='training'):
    model.train()

    src, tgt, src_mask = next(cycle())

    loss = model(src, tgt, mask=src_mask)
    loss.backward()
    print(f'{i}: {loss.item()}')

    optim.step()
    optim.zero_grad()

    ## 매 N(100)번마다 accuracy 측정
    if i != 0 and i % GENERATE_EVERY == 0:
        model.eval()
        src, _, src_mask = next(cycle())
        src, src_mask = src[:1], src_mask[:1]
        start_tokens = (torch.ones((1, 1)) * 1).long().cuda()

        sample = model.generate(src, start_tokens, ENC_SEQ_LEN, mask = src_mask)
        incorrects = (src != sample).abs().sum()

        print(f"input:  ", src)
        print(f"predicted output:  ", sample)
        print(f"incorrects: {incorrects}")

training:   0%|          | 0/1000 [00:00<?, ?it/s]

0: 3.369312047958374
1: 2.605001211166382
2: 2.2629458904266357
3: 2.067295789718628
4: 2.0331902503967285
5: 1.9952846765518188
6: 1.995937705039978
7: 1.9601572751998901
8: 1.9289021492004395
9: 1.89606511592865
10: 1.9284650087356567
11: 1.900430679321289
12: 1.8985962867736816
13: 1.8030672073364258
14: 1.8930487632751465
15: 1.8441640138626099
16: 1.844312071800232
17: 1.8722764253616333
18: 1.7966431379318237
19: 1.8276633024215698
20: 1.8489218950271606
21: 1.7717803716659546
22: 1.8022950887680054
23: 1.8424636125564575
24: 1.8115637302398682
25: 1.7822201251983643
26: 1.8293991088867188
27: 1.799150824546814
28: 1.827585220336914
29: 1.745924472808838
30: 1.73795747756958
31: 1.7325469255447388
32: 1.746811866760254
33: 1.7901296615600586
34: 1.7518354654312134
35: 1.742546796798706
36: 1.752791166305542
37: 1.7374539375305176
38: 1.7536131143569946
39: 1.7560166120529175
40: 1.757717490196228
41: 1.7151299715042114
42: 1.736454725265503
43: 1.7312383651733398
44: 1.6925609111

training:  15%|█▍        | 149/1000 [00:10<00:57, 14.86it/s]

145: 1.4499248266220093
146: 1.43039071559906
147: 1.4339624643325806
148: 1.4044880867004395
149: 1.3934632539749146
150: 1.3960987329483032
151: 1.3674575090408325
152: 1.3875889778137207
153: 1.4158600568771362
154: 1.384605050086975
155: 1.3753762245178223
156: 1.394942045211792
157: 1.3841928243637085
158: 1.3781671524047852
159: 1.3730113506317139
160: 1.3779691457748413
161: 1.3407599925994873
162: 1.4073541164398193
163: 1.3753060102462769
164: 1.3791913986206055
165: 1.3674577474594116
166: 1.3671506643295288
167: 1.3353475332260132
168: 1.3715423345565796
169: 1.325380802154541
170: 1.3434655666351318
171: 1.3494770526885986
172: 1.3193001747131348
173: 1.29862642288208
174: 1.2983230352401733
175: 1.3608295917510986
176: 1.2822078466415405
177: 1.2898211479187012
178: 1.2998143434524536
179: 1.323857307434082
180: 1.3209636211395264
181: 1.2860876321792603
182: 1.3039382696151733
183: 1.3341355323791504
184: 1.3193365335464478
185: 1.2926806211471558
186: 1.277868628501892
1

training:  36%|███▌      | 357/1000 [00:20<00:35, 18.31it/s]

...

766: 0.4216724932193756
767: 0.4411855936050415
768: 0.44861167669296265
769: 0.42293810844421387
770: 0.4225226938724518
771: 0.43896669149398804
772: 0.41206806898117065
773: 0.4184914827346802
774: 0.4181687831878662
775: 0.4530041515827179
776: 0.4488639235496521
777: 0.4151068329811096
778: 0.45143914222717285
779: 0.41532281041145325
780: 0.44381290674209595
781: 0.42842596769332886
782: 0.413967102766037
783: 0.4509863257408142
784: 0.461715430021286
785: 0.404319703578949
786: 0.42588984966278076
787: 0.4254918694496155
788: 0.4497276544570923
789: 0.429386168718338
790: 0.42234453558921814
791: 0.4265766739845276
792: 0.4177374243736267
793: 0.4229660928249359
794: 0.4104374051094055
795: 0.40400275588035583
796: 0.40139687061309814
797: 0.41006746888160706
798: 0.4080824851989746
799: 0.4034433364868164
800: 0.44160738587379456
input:   tensor([[4, 2, 3, 2, 2, 4, 4, 3]], device='cuda:0')
predicted output:   tensor([[2, 3, 4, 4, 2, 4, 2, 4]], device='cuda:0')
incorrects: 6
801

training:  98%|█████████▊| 977/1000 [00:50<00:01, 19.94it/s]

973: 0.3773336410522461
974: 0.4591202139854431
975: 0.4015272259712219
976: 0.3899928629398346
977: 0.4200470447540283
978: 0.40879687666893005
979: 0.4295567274093628
980: 0.4658403992652893
981: 0.43676117062568665
982: 0.4268330931663513
983: 0.451737642288208
984: 0.38558223843574524
985: 0.426098495721817
986: 0.38439884781837463
987: 0.42595481872558594
988: 0.43252915143966675
989: 0.4075873792171478
990: 0.4140845537185669
991: 0.4185456931591034
992: 0.4278804659843445
993: 0.3888463079929352
994: 0.4002903401851654
995: 0.3658437132835388
996: 0.39159679412841797
997: 0.3658497631549835


training: 100%|██████████| 1000/1000 [00:51<00:00, 19.43it/s]

998: 0.38177070021629333
999: 0.4084315598011017


In [6]:
print(f'Accuracy : {(len(src) - incorrects)/len(src)*100}%')

Accuracy : 87.5%
